# Tabu search

In this document, we use two examples to show how we can use the class TabuSearch, which is in the Optimpy library. By convention, we created every algorithm to find the solution where has gotten minimum value.

For this, it is necessary:

To overwrite the following functions
* get_neighbors (Required)
* encode_change (Required)
* custom the tabu list (optional)


And, to define
* the objective function $f$
* the constraints list

We will explain every step which is necessary to start to build the first search model.


### Imports

In [3]:

import sys
import os

#library_path is the path where the Optimpy library is located.
#library_path = "/home/dell/Documentos/Git_proejcts/optimizacion-con-metaheuristicas/"
library_path = "/Users/adrianamenchacamendez/Documentos/enes_morelia/papime/optimizacion-con-metaheuristicas/"
sys.path.append(os.path.abspath(library_path))


In [4]:
from optimpy.heuristic.Tabu_search import TabuSearch
from optimpy.utils.helpers import *
from pprint import pprint
import numpy as np 
import copy 

## 0/1 Knapsack problem

\begin{equation}
  \label{eq:KP}
  \begin{array}{rll}
  \text{maximize:} & f(\vec{x}) = \sum_{i=1}^{n} p_i \cdot x_{i} &  \\
  \text{where: } & g_1(\vec{x}) = \sum_{i=1}^{n} w_i \cdot x_{i}  \leq c &  \\
          &  x_i \in \{0,1\} & i\in\{1,\ldots,n\}\\
  \end{array}
\end{equation}

Consider this input:
- $n = 5$
- $p = \{5, 14, 7, 2, 23\}$
- $w = \{2, 3, 7, 5, 10\}$
- $c = 15$

Where the best solution is:
$x = [1, 1, 0, 0, 1]$ , $f(x) = 42$ and $g_{1}(x) \leq 15$

For more information about this problem, you can check our handbook.


### Objective function

In [5]:
@checkargs
def f(x : np.ndarray) -> float:
    p = np.array([5,14,7,2,23])
    return -1* np.dot(x,p)

Before starting in the next steps, we have to change the problem that we want to solve. Remember that we assume that it as a minimization problem. (That is the reason of  multiply by -1) 

### Constraints 

In [6]:
@checkargs 
def g1(x : np.ndarray) -> bool:
    w = [2,3,7,5,10]
    return np.dot(x,w) <= 15

constraints_list= [g1]

### Overwriting functions
**Note:** *It is necessary to implement get_neighbors and encode_change. If not, the algorithm does not work.* 


In [7]:
class Knapsack_solver(TabuSearch):
    @checkargs
    def __init__(self, f_ : function_type , constraints_: list):
        super().__init__(f_,constraints_)
        
    @checkargs
    def get_neighbors(self, x : np.ndarray) -> list:   
        neighbors_list = []

        for i in range(len(x)):
            x[i] ^= 1 #1
            neighbors_list+=[copy.deepcopy(x)]
            x[i] ^= 1 
            
        return neighbors_list
        
    @checkargs
    def encode_change(self, neighbor : (list,np.ndarray), x : (list,np.ndarray)) -> list: #2
        
        x_ = [None,None]
        
        for i in range(len(x)):
            if x[i] != neighbor[i]:
                return [i,neighbor[i]]
            
        return x_

* Let $x$ be a solution, where $x =[1,0,0,0,1]$, and neighbor is another solution where we flip a position of our initial solution ($x$).


```python 
neighborhood = get_neighbors(x)
print(neighborhood)

#Return a list of lists.
# [ [0,0,0,0,1],[1,1,0,0,1],[1,0,1,0,1],[1,0,0,1,1],[1,0,0,0,0] ]
```


```python
neighbor = neighborhood[0]
x = [1,0,0,0,1] 

result = encode_change(neighbor,x)
print(result)
#The result is a list, where the first element is the position of the changed bit, and the second one is the value before changing it.
#Output [0, 0]
```

In [8]:
Knapsack = Knapsack_solver(f, [g1])

init_backpack_solution = np.zeros(5,dtype=int)
'''Parameters:
    Initial solution
    Number of iterations
    Tabu time
'''
Knapsack.optimize(init_backpack_solution,30,3)
print(Knapsack)

100%|██████████| 30/30 [00:00<00:00, 981.29it/s]

Tabu search: 
 f(X) = -42 
 X = [1 1 0 0 1] 
 


In the last example, we used a small instance. Now, we will use a bigger instance.


**Note:** We recommend to use different names to avoid overlapping name problems.


In [9]:
n = 50
p = [60, 52, 90, 57, 45, 64, 60, 45, 63, 94, 44, 90, 66, 64, 32, 39, 91, 40, 73, 61, 82, 94, 39, 68, 94, 98, 80, 79, 73, 99, 49, 56, 69, 49, 82, 99, 65, 34, 31, 85, 67, 62, 56, 38, 54, 81, 98, 63, 48, 83]
w = [38, 20, 21, 21, 37, 28, 32, 30, 33, 35, 29, 32, 35, 24, 28, 29, 22, 34, 31, 36, 36, 28, 38, 25, 38, 37, 20, 23, 39, 31, 27, 20, 38, 38, 36, 28, 39, 22, 23, 22, 21, 24, 23, 33, 31, 30, 32, 30, 22, 37]
c = 870

In [10]:
@checkargs
def f(x : np.ndarray) -> float:
    global p
    return -1* np.dot(x,p)

@checkargs 
def g1(x : np.ndarray) -> bool:
    global w,c
    result = np.dot(x,w)
    g1.__doc__="{} <= {}".format(result,c)
    return result <= c

constraints_list= [g1]

In this case, we use global variables for calculating the objective function and the constraint.

### Initial solution
In this case, we start taking random objects and we try to insert them in the backpack.

In [11]:
def getInitialSolution(NumObjects=5):
    global n,p,w,c
    #Empty backpack
    x = [0 for i in range(n)]
    weight_x = 0
    
    #Random order to insert objects.
    objects = list(range(n))
    np.random.shuffle(objects)
    
    for o in  objects[:NumObjects]:
        #Check the constraint about capacity.
        if weight_x + w[o] <= c:
            x[o] = 1
            weight_x += w[o]
            
    return np.array(x)

When we create a solver instance, we have to call optimize function, which finds the best solution with the following parameters:
* The initial solution.
* The number of iterations.
* The time where we avoid to do a change  in a position (Tabu time).

In [12]:
Knapsack_2 = Knapsack_solver(f, [g1])

In [13]:
Knapsack_2.optimize(getInitialSolution,100,n//2)
print(Knapsack_2)

100%|██████████| 100/100 [00:00<00:00, 141.20it/s]

Tabu search: 
 f(X) = -2197 
 X = [0 0 1 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1
 0 0 1 1 0 1 1 0 1 1 0 0 1] 
 Constraints: 
 858 <= 870 



Now, we will check the algorithm's behavior, for this is necessary to evaluate the results for a specific number of runs. The Optimpy library has a function called **get_stats** (it is in **utils.helpers**), which returns a dictionary with some statistics of the runs. The parameters are:
* The solver.
* The number of iterations to evaluate model.
* The arguments, this should be a tuple.

In [14]:
args = (getInitialSolution,500,n//2)
statistics = get_stats(Knapsack_2, 21, args)

100%|██████████| 500/500 [00:02<00:00, 181.37it/s]


In [13]:
pprint(statistics)

{'Best solution': {'f': -2301,
                   'x': array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1])},
 'Mean': -2251.9523809523807,
 'Standard deviation': 34.28935165890729,
 'Worst solution': {'f': -2174,
                    'x': array([0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1])}}


## Travelling salesman problem


\begin{equation}
    \label{eq:TSP}
    \begin{array}{rll}
    \text{minimize:} & f(x) = d(x_n, x_1) + \sum_{i=1}^{n-1} d(x_i, x_{i+1}) &  \\
    \text{such that: } & x_i \in \{1,2,\cdots,n\} & \\
    \end{array}
\end{equation}

where $d(x_i, x_j)$ is the distance from city $x_i$ to city $x_j$, $n$ is the number of cities and $x$ is a permutation of $n$ cities.

In [14]:
import random 

In [15]:
num_cities = 10
iterations = 100
dist_matrix = \
[\
[0,49,30,53,72,19,76,87,45,48],\
[49,0,19,38,32,31,75,69,61,25],\
[30,19,0,41,98,56,6,6,45,53],\
[53,38,41,0,52,29,46,90,23,98],\
[72,32,98,52,0,63,90,69,50,82],\
[19,31,56,29,63,0,60,88,41,95],\
[76,75,6,46,90,60,0,61,92,10],\
[87,69,6,90,69,88,61,0,82,73],\
[45,61,45,23,50,41,92,82,0,5],\
[48,25,53,98,82,95,10,73,5,0],\
]


In [16]:
@checkargs
def f_salesman(x : np.ndarray) -> float:
    global dist_matrix
    total_dist = 0
    for i in range(1,len(x)):
        u,v = x[i], x[i-1]
        total_dist+= dist_matrix[u][v]
    total_dist += dist_matrix[x[-1]][0]
    return total_dist

In [17]:
@checkargs
def g_salesman(x : np.ndarray) -> bool:
    """
    Xi in {1,2, ... , N}
    """
    size = len(x)
    size_ = len(np.unique(x))
    return size == size_

Now, we will change the tabu list's structure. For this, it is necessary to implement a new class with the following functions:

* reset
* update
* push_front
* find 

### Description
- **reset:** clear the current structure used to store the items.
- **update:** remove the items where the timer is finished and update the timer for every item in the data structure.
- **push_front:** add new items to the custom data structure.
- **find:** search if the item is in the data structure.

In [18]:
class Tabu_Salesman_list:
    def __init__(self,timer):
        self.__TB = {}
        self.timer = timer
    
    def reset(self,timer) -> None:
        self.__TB = {}
        self.timer = timer
        
    def update(self) -> None:
        to_pop = []
        for key in self.__TB:
            if self.__TB[key]-1 == 0:
                to_pop.append(key)
            else:
                self.__TB[key]-=1
        for key in to_pop:
            self.__TB.pop(key)
        
    @checkargs
    #x has [p,v,step], we are only interested in v (value)
    def push_front(self, x : list ) -> None:
        self.__TB[x[1]] = self.timer
        
    @checkargs
    def find(self, x : list) -> bool:
        return x[1] in self.__TB
        

In [19]:
class TravellingSalesman_solver(TabuSearch):

    def __init__(self, f_ : function_type , constraints_: list, TabuStorage):
        super().__init__(f_,constraints_,TabuStorage)
        
    @checkargs
    def get_neighbors(self, x : np.ndarray) -> list: 
        
        neighbors_list = []
        
        ind = random.randint(1,len(x)-1)
        while  self.TL.find([-1,x[ind]]):
            ind = random.randint(1,len(x)-1)
        v = x[ind]
        x_tmp = list(x[v != x])
        for i in range(1, len(x)):
            if ind == i:
                continue
            neighbors_list += [ x_tmp[:i] + [v] + x_tmp[i:]]
            
        return neighbors_list

        
    @checkargs
    def encode_change(self, neighbor : (list,np.ndarray), x : (list,np.ndarray)) -> list: #2
        
        x_p ={x[i] : i for i in range(len(x))}
        n_p = {neighbor[i]: i for i in range(len(x))}
        ind = -1
        max_dist = -1
        value = -1
        for i in range(1, len(x)):
            v = x[i]
            dist = abs(x_p[v] - n_p[v])
            if dist > max_dist:
                ind = i
                max_dist = dist
                value = v
       
        return [ind , value]

### Initial solution
In this case, we create the initial solution using a greedy strategy.

In [20]:
def getInitialSolutionTS(distance_matrix, total_cities):
    Solution = [0]
    remaining_cities  = list(range(1,total_cities))
    
    while len(remaining_cities) != 0:
        from_ =Solution[-1] 
        to_ = remaining_cities[0]
        dist = distance_matrix[from_][to_]
        
        for i in range(1, len(remaining_cities)):
            distance = distance_matrix[from_][remaining_cities[i]]
            if distance < dist:
                to_ = remaining_cities[i]
                dist = distance
        Solution.append(to_)
        ind = remaining_cities.index(to_)
        remaining_cities.pop(ind)
    return Solution

In [21]:
TravellingSalesman = TravellingSalesman_solver(f_salesman,[g_salesman],Tabu_Salesman_list(num_cities//2))
init_path = np.array(getInitialSolutionTS(dist_matrix,num_cities))
print("Initialize search with this initial point {} \n f(x) = {}".format(init_path, f_salesman(init_path)))




Initialize search with this initial point [0 5 3 8 9 6 2 7 1 4] 
 f(x) = 271


In [22]:
TravellingSalesman.optimize(init_path, iterations, num_cities//2)
print(TravellingSalesman)

100%|██████████| 100/100 [00:00<00:00, 508.72it/s]

Tabu search: 
 f(X) = 248 
 X = [0 5 3 8 9 6 2 7 4 1] 
 Constraints: 
 
    Xi in {1,2, ... , N}
     



In [23]:
args = (init_path, iterations, num_cities//2)
statistics = get_stats(TravellingSalesman, 30, args)


100%|██████████| 100/100 [00:00<00:00, 645.79it/s]


In [24]:

pprint(statistics)

{'Best solution': {'f': 248, 'x': array([0, 5, 3, 8, 9, 6, 2, 7, 4, 1])},
 'Mean': 248.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': 248, 'x': array([0, 5, 3, 8, 9, 6, 2, 7, 4, 1])}}
